## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#wall2 = gscene.create_safe(GEOTYPE.BOX, "wall2", "base_link", (0.6,0.8,0.01), (0.5,0,0),
#                            rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#                            rpy=np.random.rand(3)*np.pi/16, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

In [12]:
pscene.add_handle_axis('girpper', pscene.actor_dict['grip0'])

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [13]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [14]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType, PlannerConfig

gtimer = GlobalTimer.instance()
gtimer.reset()

In [15]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, motion_filters=[GraspChecker(pscene)])

##### PDDLStream
* pick & place
* currently only *single robot* is supported
* only colliding geometries are transfered to PDDLStream(pybullet). make sure collision=True for all subjects and actors.

In [16]:
from pkg.planning.pddlstream.plan_rnb import *

### solve in pddlstream general

In [17]:
domain_path = os.path.join(RNB_PLANNING_DIR, "src", "pkg", "planning", "pddlstream", "domain")

In [18]:
pscene, mplan, ROBOT_NAME, TOOL_NAME, HOME_POSE, goal_pairs, initial_state = \
    pscene, mplan, 'indy0', 'grip0', crob.home_pose, None, initial_state

TIMEOUT_MOTION=0.5
MAX_TIME=30
MAX_ITER=100
MAX_SKELETONS=10
GRASP_SAMPLE=30
STABLE_SAMPLE=50
SHOW_STATE=False
SEARCH_SAMPLE_RATIO=10
use_pybullet_gui=True
USE_MOVEIT_IK=True


TIMED_COMPLETE=False
VERBOSE=False
IK_TRY_NUM=10
IK_TIMEOUT_SINGLE=0.01

                
                
gtimer = GlobalTimer.instance()
gscene = pscene.gscene
#     checkers_ik = [checker for checker in mplan.motion_filters if checker.BEFORE_IK]
checkers_ik = [checker for checker in mplan.motion_filters]
# mplan.motion_filters = [checker for checker in mplan.motion_filters if not checker.BEFORE_IK]
mplan.motion_filters = []
checkers_ik_names = [checker.__class__.__name__ for checker in checkers_ik]
checkers_mp_names = [checker.__class__.__name__ for checker in mplan.motion_filters]
connect_notebook(use_gui=use_pybullet_gui)
urdf_pybullet_path = copy_meshes(gscene)
reset_pybullet()
robot_body, body_names, movable_bodies = pscene_to_pybullet(
    pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
print('Objects:', body_names)
saver = WorldSaver()
mplan.reset_log(True)


pscene, robot, initial_state = pscene, robot_body, initial_state

body_names=body_names
Q_init=HOME_POSE
# goal_pairs=goal_pairs
movable=movable_bodies
checkers_ik=checkers_ik
tool_name=TOOL_NAME
tool_link_name=pscene.actor_dict[TOOL_NAME].geometry.link_name
mplan=mplan
timeout=TIMEOUT_MOTION
grasp_sample=GRASP_SAMPLE
stable_sample=STABLE_SAMPLE
show_state=SHOW_STATE
USE_MOVEIT_IK=USE_MOVEIT_IK
TIMED_COMPLETE=TIMED_COMPLETE
IK_TRY_NUM=IK_TRY_NUM
IK_TIMEOUT_SINGLE=IK_TIMEOUT_SINGLE
                                            
print("================ MAKE PROBLEM ======================")
print("IK checkers: {}".format([checker.__class__.__name__ for checker in checkers_ik]))
print("MP checkers: {}".format([checker.__class__.__name__ for checker in mplan.motion_filters]))
print("timeout motion : {}".format(timeout))
print("====================================================")
#assert (not are_colliding(tree, kin_cache))
assert tool_link_name is not None, "tool_link_name should be passed to pddlstream_from_problem"
assert tool_name is not None, "tool_name should be passed to pddlstream_from_problem"
assert mplan is not None, "mplan should be passed to pddlstream_from_problem"

# if len(checkers_ik)==0 and len(mplan.motion_filters)==0:
#     print("No predictors are assigned. Automatically set TIMED_COMPLETE=False")
#     TIMED_COMPLETE = False

[Pybullet] Load urdf from /home/jb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
[WARING] non-object subject not implemented for now
('Objects:', {1L: 'goal', 2L: 'gripper', 3L: 'floor', 4L: 'wall', 5L: 'obstacle', 6L: 'box1'})
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker']
MP checkers: []
timeout motion : 0.5


In [19]:
domain_pddl = read(get_file_path(domain_path+"/dummy", 'domain_general.pddl'))
stream_pddl = read(get_file_path(domain_path+"/dummy", 'stream_general.pddl'))
constant_map = {}

print('Robot:', robot)
set_configuration(robot, Q_init)
conf = BodyConf(robot, get_configuration(robot))
conf_temp = BodyConf(robot,[ 1.42505094, -0.49137475, -2.19144837, -0.38963825, -0.463891,   -1.3667626 ])
conf_f = BodyConf(3L, [])
init = [('CanMove',),
        ('HandEmpty',),
        ('Conf', robot, conf),
        ('AtConf', robot, conf)]

initial_state = pscene.initialize_state(Q_init)
init_btf = initial_state.binding_state['box1']
init_binding = Binding(init_btf)

fixed = get_fixed(robot, movable)
print('Movable:', movable)
print('Fixed:', fixed)
for body in movable:
    pose = BodyPose(body, get_pose(body))
    init += [('Actor', robot, 2L),
             ('Actor', 3L, 1L),
             ('Actor', 3L, 3L),
             ('Robot', robot),
             ('Robot', 3L),
             ('Conf', 3L, conf_f),
             ('AtConf', 3L, conf_f),
             ('Subject', 6L),
             ('Pose', body, pose),
             ('Binding', 6L, 3L, init_binding),
             ('BindingPose', body, 3L, init_binding, pose),
             ('AtPose', body, pose)
             ]
print('initial pose')
print(pose.value)
body_subject_map = make_body_subject_map(pscene, body_names)
print('body subject map')
print(body_subject_map)
body_actor_map = make_body_actor_map(pscene, body_names)
print('body actor map')
print(body_actor_map)
subject_body_map = {sj.oname: bid for bid, sj in body_subject_map.items()}
print('subject body map')
print(subject_body_map)
actor_body_map = {at.name: bid for bid, at in body_actor_map.items()}
print('actor body map')
print(actor_body_map)
# update_grasp_info({tool_name: GraspInfo(
#     lambda body: sample_grasps(body_subject_map=body_subject_map, body=body, actor=actor,
#                                sample_count=grasp_sample, show_state=show_state),
#     approach_pose=Pose(APPROACH_VEC))})
if USE_MOVEIT_IK:
    ik_kwargs = dict(mplan=mplan, timeout_single=IK_TIMEOUT_SINGLE)
else:
    ik_kwargs = {}
ik_fun = get_ik_fn_general_rnb(
    pscene, actor_body_map, mplan, fixed=fixed)

stream_map = {
    'sample-binding': from_gen_fn(get_binding_gen_rnb(body_subject_map, body_actor_map)),
    'sample-transform': from_fn(get_transform_gen_rnb(pscene, body_actor_map, subject_body_map)),
    'inverse-kinematics': from_fn(ik_fun),

    'plan-motion': from_fn(get_general_motion_gen_rnb(mplan, body_subject_map, robot,
                                                        tool=pscene.actor_dict[tool_name],
                                                        tool_link=tool_link_name, timeout=timeout,
                                                        show_state=show_state,
                                                        approach_vec=None)),
    'test-cfree-binding-binding': from_test(get_cfree_binding_binding_test_rnb()),
    'test-cfree-conf-pose': from_test(get_cfree_conf_pose_test_rnb()),
    'test-equal-pose-value': from_test(get_equal_pose_value_test_rnb())
}

('Robot:', 0L)
('Movable:', [6L])
('Fixed:', [1, 2, 3, 4, 5])
initial pose
((0.30000001192092896, 0.4000000059604645, 0.03099999949336052), (0.0, 0.0, 0.0, 1.0))
body subject map
{3L: <pkg.planning.constraint.constraint_subject.SweepLineTask object at 0x7f4710fc2250>, 6L: <pkg.planning.constraint.constraint_subject.CustomObject object at 0x7f4710fd7150>}
body actor map
{1L: <pkg.planning.constraint.constraint_actor.PlacePlane object at 0x7f4710fb64d0>, 2L: <pkg.planning.constraint.constraint_actor.Gripper2Tool object at 0x7f4710fc20d0>, 3L: <pkg.planning.constraint.constraint_actor.PlacePlane object at 0x7f4710fb6510>}
subject body map
{'box1': 6L, 'sweep': 3L}
actor body map
{'grip0': 2L, 'goal': 1L, 'floor': 3L}


In [20]:

# binding_f = Binding(body_subject_map[6L].binding)
# init += [('Binding', 6L, 3L, binding_f),
#          ('BindingPose', 6L, 3L, binding_f, pose),]

# goal = ('and', ('AtConf', robot, conf)) \
#        + tuple([
#            ('On', subject_body_map[sname], actor_body_map[aname]) for sname, aname in goal_pairs
#        ])
# goal = ('AtConf', robot, conf_temp)
goal = ('On', subject_body_map['box1'], actor_body_map['goal'])
# goal = ('and', ('AtPose', 6L, pose)) \
#        + tuple([
#            ('On', subject_body_map['box1'], actor_body_map['goal']) for sname, aname in goal_pairs
#        ])
# goal = ('FindConf', 0L, pose, 6L, 2L)
# goal = ('FindConf', pose, 6L, 2L)
# goal = ('FindBinding', 1L, 6L)
# goal = ('FindBP', 6L, 2L)
# goal = ('MoveToConf', robot, pose, 6L, 2L)
# goal = ('GoToConf', robot, pose, 6L, 2L)
        # ('Holding', body),
        # ('On', body, fixed[2]),
        # ('Cleaned', body),
        #             ('Cooked', body),
APPROACH_VEC = 0.00 * Point(z=-1)
actor = pscene.actor_dict[tool_name]

reset_checker_cache()

problem, ik_fun = PDDLProblem(domain_pddl, constant_map, stream_pddl, stream_map, init, goal), ik_fun

# GlobalLogger.instance()["ik_fun"] = ik_fun
# GlobalLogger.instance()["problem"] = problem
_, _, _, stream_map, init, goal = problem
print('Init:', init)
print('Goal:', goal)
print('Streams:', str_from_object(set(stream_map)))
ik_fun.checkout_count = 0
ik_fun.fail_count = 0
ik_fun.pass_count = 0
with Profiler():
    with LockRenderer(lock=not True):
        gtimer.tic("plan")
        solution = solve(problem, algorithm='adaptive',
                         unit_costs=False, success_cost=INF, max_time=MAX_TIME, max_iterations=MAX_ITER,
                         max_skeletons=MAX_SKELETONS, search_sample_ratio=SEARCH_SAMPLE_RATIO, verbose=VERBOSE)
        gtimer.toc("plan") / gtimer.scale
        saver.restore()
print_solution(solution)
plan, cost, evaluations = solution
# GlobalLogger.instance()["solution"] = solution
res = not any(plan is status for status in [None, False])

move_num = len(plan) if res else 0
pre_motion_checks = mplan.result_log["pre_motion_checks"]
plan_try = len(pre_motion_checks)
planning_log = mplan.result_log["planning"]
plan_num = len(planning_log)
fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
elapsed = SolutionStore.last_log['run_time']

log_dict = {"plan_time": elapsed, "length": move_num,
            "IK_tot": ik_fun.checkout_count + ik_fun.pass_count + ik_fun.fail_count,
            "IK_count": ik_fun.pass_count + ik_fun.fail_count, "failed_IKs": ik_fun.fail_count,
            "MP_tot": plan_try, "MP_count": plan_num, "failed_MPs": fail_num,
            "success": res, "body_names": body_names, "plan": plan,
            "pre_motion_checks": pre_motion_checks, "planning_log": planning_log,
            "checkers_ik": checkers_ik_names, "checkers_mp":checkers_mp_names,
            "time_log": gtimer.timelist_dict if gtimer.stack else gtimer.time_dict}

('Init:', [('CanMove',), ('HandEmpty',), ('Conf', 0L, q0), ('AtConf', 0L, q0), ('Actor', 0L, 2L), ('Actor', 3L, 1L), ('Actor', 3L, 3L), ('Robot', 0L), ('Robot', 3L), ('Conf', 3L, q2), ('AtConf', 3L, q2), ('Subject', 6L), ('Pose', 6L, p0), ('Binding', 6L, 3L, b0), ('BindingPose', 6L, 3L, b0, p0), ('AtPose', 6L, p0)])
('Goal:', ('On', 6L, 1L))
('Streams:', '{inverse-kinematics, plan-motion, sample-binding, sample-transform, test-cfree-binding-binding, test-cfree-conf-pose, test-equal-pose-value}')
Warning! All actions have no cost. Recommend setting unit_costs=True

Iteration: 1 | Complexity: 0 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 16 | Eager Calls: 0 | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Total Time: 0.000
Attempt: 1 | Results: 4 | Depth: 0 | Success: False | Time: 0.011
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 0 to 1

Iteration: 2 | Complexity: 1 | Skeletons: 0 | Skeleton Queue: 0 | Disab

3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b69
{'goal': {'w': -0.37627575013490944}, 'bottom': {'w': -2.31275509645618}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.35758722 -0.9338798   0.          0.3       ]
 [ 0.9338798  -0.3575872   0.         -0.39999998]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b70
{'goal': {'w': -1.509821590926009}, 'bottom': {'w': -0.43566795869330566}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.47647655  0.8791872   0.          0.3       ]
 [-0.8791872   0.47647655  0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf bo

sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b140
{'goal': {'w': 2.5373141294621853}, 'bottom': {'w': -0.9688994982237569}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.934259    0.3565952   0.          0.3       ]
 [-0.35659516 -0.934259    0.         -0.4       ]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b141
{'goal': {'w': -3.015056452570169}, 'bottom': {'w': -1.8539413650487075}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.398317    0.9172479   0.          0.3       ]
 [-0.9172479   0.39831704  0.         -0.4       ]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan ge

3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b211
{'goal': {'w': 0.34921440530369097}, 'bottom': {'w': -2.187452882246477}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.8225445  -0.56870055  0.          0.30000007]
 [ 0.5687006  -0.8225445   0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b212
{'goal': {'w': -2.2177732819872813}, 'bottom': {'w': -1.3663966880141467}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.6589483   0.75218827  0.          0.29999998]
 [-0.7521883   0.6589483   0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.  

sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b258
{'goal': {'w': 0.16446350712890512}, 'bottom': {'w': -2.728665644695748}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.9692914  -0.2459149   0.          0.30000004]
 [ 0.2459149  -0.9692914   0.         -0.39999998]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b259
{'goal': {'w': 0.8859975459861911}, 'bottom': {'w': -0.5980343801482588}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.08665559 -0.9962383   0.          0.3       ]
 [ 0.99623835  0.08665557  0.         -0.40000007]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan ge

...

{'goal': {'w': -0.5059218839967303}, 'bottom': {'w': 0.43507685900201265}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.5889812  0.8081467  0.         0.3      ]
 [-0.8081467  0.5889812  0.        -0.4      ]
 [ 0.         0.         1.         0.031    ]
 [ 0.         0.         0.         1.       ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b4964
{'goal': {'w': 0.2640054560264251}, 'bottom': {'w': 1.4000645365992392}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.42117208  0.9069807   0.          0.3       ]
 [-0.90698075  0.42117208  0.         -0.40000007]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundanc

b5031
{'goal': {'w': -0.3724963830215704}, 'bottom': {'w': -2.567042779297462}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.5840832  -0.81169385  0.          0.29999998]
 [ 0.81169385 -0.5840832   0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b5032
{'goal': {'w': -2.798033882381778}, 'bottom': {'w': -1.2742837573655053}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.04702889  0.9988935   0.          0.3       ]
 [-0.99889356  0.04702889  0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample 

plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b5096
{'goal': {'w': 1.863700240707514}, 'bottom': {'w': 2.4316996462134517}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.84297884  0.53794664  0.          0.3       ]
 [-0.53794664  0.84297884  0.         -0.40000004]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b5097
{'goal': {'w': -3.0660822563404957}, 'bottom': {'w': 1.0025644203148047}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.6001929  -0.79985523  0.          0.30000004]
 [ 0.79985523 -0.6001929   0.         -0.39999998]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.        

BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[ 0.10521886 -0.9944491   0.          0.3       ]
 [ 0.9944491   0.10521887  0.         -0.4       ]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b5160
{'goal': {'w': 0.4455978101016367}, 'bottom': {'w': -1.9871755907901405}}
BindingChain(subject_name='box1', handle_name='bottom', actor_name='goal', actor_root_gname='goal')
[[-0.759131   -0.6509379   0.          0.3       ]
 [ 0.65093786 -0.759131    0.         -0.39999998]
 [ 0.          0.          1.          0.031     ]
 [ 0.          0.          0.          1.        ]]
ik_fn print conf body val
3
[]
plan general motion conf2.values
3
[]
robot not 0L
3
sample bd: binding, redundancy, .btf.chain, .btf.T_lao
b5161
{'goal': {'w': -2.09944298

         30883472 function calls (30195980 primitive calls) in 30.528 seconds

   Ordered by: internal time
   List reduced from 1277 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   135822    6.280    0.000    6.280    0.000 /usr/local/lib/python2.7/dist-packages/zmq/sugar/socket.py:357(send)
   135718    1.083    0.000    8.422    0.000 /usr/local/lib/python2.7/dist-packages/ipykernel/iostream.py:382(write)
    38810    0.505    0.000    1.267    0.000 /home/jb/Projects/pddlstream/pddlstream/algorithms/../../downward/builds/release/bin/translate/translate.py:67(translate_strips_conditions_aux)
211255/27077    0.478    0.000    0.930    0.000 /home/jb/Projects/pddlstream/pddlstream/language/conversion.py:81(substitute_expression)
    82067    0.443    0.000    0.634    0.000 /home/jb/Projects/pddlstream/pddlstream/algorithms/../../downward/builds/release/bin/translate/pddl/conditions.py:288(instantiate)
   355655    0.438    0.0

In [21]:
play_general_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
               body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)

In [22]:
pscene.gscene.show_pose([1.42505094, -0.49137475, -2.19144837, -0.38963825, -0.463891, -1.3667626])

In [36]:
pscene.gscene.show_pose([1.30953555, -1.90757443, -1.12179683, 0, 3.02937128, -1.30953372])

In [27]:
play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
               body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)

In [27]:
gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.35000000148821875, -0.1864999923833766, 0.3614999838183085), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)